In [101]:
import math
import matplotlib.pyplot as plt
import matplotlib.lines as mpl_lines
import numpy as np

%matplotlib widget

# code modified to change negative values to positive ones
def lmap(f,x):
    return list(map(f,x))

# code modified to change negative values to positive ones
def lfilter(f,x):
    return list(filter(f,x))

def loadCSV(path, x_id=0, y_id=1):
    with open(path) as inp:
        lines = inp.read().split("\n")[1:-2]
        rows = lmap(lambda l: lmap(float, l.strip().split(",")), lines)

        return {"x": lmap(lambda r: r[x_id], rows), "y": lmap(lambda r: r[y_id], rows)}

In [102]:
def eval_state(distance_cm, omega_driving, k_omegs):

    # print(f"./runs/d={distance_cm:.2f}_o={omega_driving:.2f}_k={k_omegs:.3f}.csv")
    dat = loadCSV(f"./runs/d={distance_cm:.2f}_o={omega_driving:.2f}_k={k_omegs:.3f}.csv")

    conv_len = 25
    dat_smooth = {
        "x": dat["x"][0: -conv_len + 1],
        "y": np.convolve(dat["y"], np.ones(conv_len) * 1/conv_len, "valid")
    }

    d_dat_smooth = {
        "x": dat_smooth["x"][:-1],
        "y": np.diff(dat_smooth["y"]) / (dat_smooth["x"][1] - dat_smooth["x"][0])
    }

    # plt.figure()
    # plt.plot(dat["x"], dat["y"])
    # plt.plot(dat_smooth["x"], dat_smooth["y"])
    # plt.plot(d_dat_smooth["x"], d_dat_smooth["y"])

    # print("Avg: ", np.average(dat["y"]))
    # print("Dev: ", np.average(np.abs(dat_smooth["y"] - np.average(dat["y"]))))
    # print("Switch dir: ", any(lmap(lambda l: l>0, dat["y"])))
    # print("Avg deriv.: ", np.average(d_dat_smooth["y"]))

    return (
        np.average(dat["y"]),
        np.average(np.abs(dat_smooth["y"] - np.average(dat["y"]))),
        any(lmap(lambda l: l>0, dat["y"])),
        np.average(d_dat_smooth["y"])
    )

In [103]:
open("./results.csv", "w").close()
with open("./results.csv", "a+") as wr:
    wr.write(
        "d, omega_{driving}, omega_{driven}/omega_{driving}, Avg, Dev, Switch dir., Avg deriv.\n"
    )

    dist = 7
    while dist < 20:
        o1 = 10
        while o1 <= 10:
            k = 0
            while k <= 7:
                try:
                    (a, b, c, d) = eval_state(dist, o1, k)
                    wr.write(f"{dist},{o1},{k},{a},{b},{c},{d}\n")
                except Exception as e:
                    # print(e)
                    pass

                if k < 2:
                    k = k + 0.01
                elif k < 4:
                    k = k + 0.025
                else:
                    k = k + 0.1
                k = round(k * 1000) / 1000

            o1 += 0.1
            o1 = round(o1 * 100) / 100

        if dist < 10:
            dist = dist + 0.1
        elif dist < 14:
            dist = dist + 0.25
        else:
            dist = dist + 0.5

        d = round(d * 100) / 100